# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [18]:
import findspark
findspark.init('C:\Spark\spark-3.1.1-bin-hadoop2.7')

In [19]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, VectorAssembler, Normalizer, StandardScaler
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [20]:
! python --version

Python 3.7.10


In [21]:
! echo %PYSPARK_PYTHON%

C:\Python3\


In [22]:
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

In [23]:
# FOR RUNNING ON A EC2 CLUSTER 
#event_data_s3n_small = "s3n://udacity-dsnd/sparkify/mini_sparkify_event_data.json"
#event_data_s3n_full = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
event_data_local = "data/mini_sparkify_event_data.json"
df = spark.read.json(event_data_local)
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [24]:
df.where(df.userId == 30).select("artist").head(5)

[Row(artist='Martha Tilston'),
 Row(artist='Adam Lambert'),
 Row(artist='Daft Punk'),
 Row(artist='Starflyer 59'),
 Row(artist=None)]

In [25]:
df.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [26]:
print("UserId = null")
len(df.where(df.userId.isNull()).collect())

UserId = null


0

In [27]:
print("UserId = empty")
len(df.where(df.userId == "").collect())

UserId = empty


8346

In [28]:
print("SessionId = null")
len(df.where(df.sessionId.isNull()).collect())

SessionId = null


0

In [29]:
print("SessionId = empty")
len(df.where(df.sessionId == "").collect())

SessionId = empty


0

In [30]:
print("Remove rows with empty userIds.")
print("Processing.")
df = df.where(df.userId != "")
print("Done.")
print("Check how many empty userIds are left:")
len(df.where(df.userId == "").collect())

Remove rows with empty userIds.
Processing.
Done.
Check how many empty userIds are left:


0

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [31]:
df.groupBy("page").count().show()

+--------------------+------+
|                page| count|
+--------------------+------+
|              Cancel|    52|
|    Submit Downgrade|    63|
|         Thumbs Down|  2546|
|                Home| 10082|
|           Downgrade|  2055|
|         Roll Advert|  3933|
|              Logout|  3226|
|       Save Settings|   310|
|Cancellation Conf...|    52|
|               About|   495|
|            Settings|  1514|
|     Add to Playlist|  6526|
|          Add Friend|  4277|
|            NextSong|228108|
|           Thumbs Up| 12551|
|                Help|  1454|
|             Upgrade|   499|
|               Error|   252|
|      Submit Upgrade|   159|
+--------------------+------+



In [32]:
# Get users that churned
churned_users = df.where(df.page == "Cancellation Confirmation")

churned_users.select("userId").head(5)

[Row(userId='18'),
 Row(userId='32'),
 Row(userId='125'),
 Row(userId='105'),
 Row(userId='17')]

In [33]:
churn = udf(lambda x: 1, IntegerType())
churned_users = churned_users.withColumn("Churn", churn(churned_users.userId))
churned_users.head()

Py4JJavaError: An error occurred while calling o94.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 1 times, most recent failure: Lost task 0.0 in stage 18.0 (TID 246) (DESKTOP-FTE9C29 executor driver): java.io.IOException: Cannot run program "C:\Python3\": CreateProcess error=5, Zugriff verweigert
	at java.lang.ProcessBuilder.start(Unknown Source)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:165)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:70)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.apache.spark.rdd.RDD.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.